In [1]:
#decoding the data
import numpy as np
import tensorflow as tf
import helpers

import os



SEPCTURAL_SAMPLES = 40

#FEATURE_DIM = SEPCTURAL_SAMPLES*6*2   # This data set = 30
FEATURE_DIM=30



#INTER_DIM = 120  #60
INTER_DIM=30


#OUT_DIM = 6#len(idDict)  #66
OUT_DIM = 66


#WIDE = 20  #time step  #40
WIDE=40





BATCH_SIZE = 16
TOTAL_ITER_NUM = 1000000000

In [2]:
import pdb
def read_audio_csv(filename_queue):
	reader = tf.TextLineReader()
	key, value = reader.read(filename_queue)

	defaultVal = [[0.] for idx in range(WIDE*FEATURE_DIM + OUT_DIM)]

	fileData = tf.decode_csv(value, record_defaults=defaultVal)

	features = fileData[:WIDE*FEATURE_DIM]
  
	features = tf.reshape(features, [WIDE, FEATURE_DIM])
	labels = fileData[WIDE*FEATURE_DIM:] 

	return features, labels

def input_pipeline(filenames, batch_size, shuffle_sample=True, num_epochs=None):
	filename_queue = tf.train.string_input_producer(filenames, shuffle=shuffle_sample)

	# filename_queue = tf.train.string_input_producer(filenames, num_epochs=TOTAL_ITER_NUM*EVAL_ITER_NUM*10000000, shuffle=shuffle_sample)
	example, label = read_audio_csv(filename_queue)
	min_after_dequeue = 2000#int(0.4*len(csvFileList)) #1000
	capacity = min_after_dequeue + 3 * batch_size
	if shuffle_sample:
		example_batch, label_batch = tf.train.shuffle_batch(
			[example, label], batch_size=batch_size, num_threads=16, capacity=capacity,
			min_after_dequeue=min_after_dequeue)
	else:
		example_batch, label_batch = tf.train.batch(
			[example, label], batch_size=batch_size, num_threads=16)
	return example_batch, label_batch

In [3]:
csvFileList = []

csvDataFolder1 = os.path.join('N_Z')
#csvDataFolder1 = os.path.join('Data_V1')

print(csvDataFolder1)
orgCsvFileList = os.listdir(csvDataFolder1)

len(orgCsvFileList)


N_Z


76

In [4]:
for csvFile in orgCsvFileList:
	if csvFile.endswith('.csv'):
		csvFileList.append(os.path.join(csvDataFolder1, csvFile))

In [5]:
len(csvFileList)

76

In [6]:
batch_size=16
batch_feature, batch_label = input_pipeline(csvFileList, batch_size)
#print(batch_feature)
#pdb.set_trace() 
#print(batch_label)

In [7]:
sensor_inputs = tf.expand_dims(batch_feature, axis=3)

In [8]:
sensor_inputs

<tf.Tensor 'ExpandDims:0' shape=(16, 40, 30, 1) dtype=float32>

In [9]:
output_keep_prob=0.5
train=False

gru_cell1 = tf.contrib.rnn.GRUCell(128)
if train:
    gru_cell1 = tf.contrib.rnn.DropoutWrapper(gru_cell1, output_keep_prob=0.5)

gru_cell2 = tf.contrib.rnn.GRUCell(128)
if train:
    gru_cell2 = tf.contrib.rnn.DropoutWrapper(gru_cell2, output_keep_prob=0.5)

Instructions for updating:
Use the retry module or similar alternatives.


In [10]:
used = tf.sign(tf.reduce_max(tf.abs(batch_feature), reduction_indices=2))

In [11]:
#print(batch_feature)

In [12]:
length = tf.reduce_sum(used, reduction_indices=1) #(BATCH_SIZE)

In [13]:
length

<tf.Tensor 'Sum:0' shape=(16,) dtype=float32>

In [14]:
length = tf.cast(length, tf.int64)
length

<tf.Tensor 'Cast:0' shape=(16,) dtype=int64>

In [15]:
sensor_inputs

<tf.Tensor 'ExpandDims:0' shape=(16, 40, 30, 1) dtype=float32>

In [16]:
#cell = tf.contrib.rnn.MultiRNNCell([gru_cell1, gru_cell2])
cell = tf.contrib.rnn.MultiRNNCell([gru_cell1,gru_cell2])

In [17]:
init_state = cell.zero_state(BATCH_SIZE, tf.float32)

In [18]:
cell_output, final_stateTuple = tf.nn.dynamic_rnn(cell, batch_feature, sequence_length=length, initial_state=init_state, time_major=False)

In [19]:
cell_output

<tf.Tensor 'rnn/transpose_1:0' shape=(16, 40, 128) dtype=float32>

In [20]:
inputs=batch_feature
mask = tf.sign(tf.reduce_max(tf.abs(inputs), reduction_indices=2, keepdims=True))
print(mask)
mask = tf.tile(mask, [1,1,128]) # (BATCH_SIZE, WIDE, INTER_DIM)
avgNum = tf.reduce_sum(mask, reduction_indices=1) #(BATCH_SIZE, INTER_DIM)
print mask

Tensor("Sign_1:0", shape=(16, 40, 1), dtype=float32)
Tensor("Tile:0", shape=(16, 40, 128), dtype=float32)


In [21]:
print(mask)
print(avgNum )

Tensor("Tile:0", shape=(16, 40, 128), dtype=float32)
Tensor("Sum_1:0", shape=(16, 128), dtype=float32)


In [22]:
layers = tf.contrib.layers 
sum_cell_out = tf.reduce_sum(cell_output*mask, axis=1, keep_dims=False)
avg_cell_out = sum_cell_out/avgNum

logits = layers.fully_connected(avg_cell_out, OUT_DIM, activation_fn=None, scope='output')


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [23]:
predict = tf.argmax(logits, axis=1)
predict
batchLoss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=batch_label)
loss = tf.reduce_mean(batchLoss)
t_vars = tf.trainable_variables()

regularizers = 0.
for var in t_vars:
	regularizers += tf.nn.l2_loss(var)
loss += 5e-4 * regularizers


discOptimizer = tf.train.AdamOptimizer(
		learning_rate=1e-4, 
		beta1=0.5,
		beta2=0.9
	).minimize(loss, var_list=t_vars)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [24]:
import matplotlib.pyplot as plt


In [ ]:
tf.initialize_all_variables()

saver = tf.train.Saver()

with tf.Session() as sess:
	tf.train.start_queue_runners()
	sess.run(tf.local_variables_initializer())
	tf.global_variables_initializer().run()

	coord = tf.train.Coordinator()
	threads = tf.train.start_queue_runners(coord=coord)

	for iteration in xrange(1000000000):
		print("Iteration NUmber :", iteration)        
		_, lossV, _trainY, _predict = sess.run([discOptimizer, loss, batch_label, predict])
        
		        
		print(lossV)
		_label = np.argmax(_trainY, axis=1)
		_accuracy = np.mean(_label == _predict)        
		print(_accuracy)
		coord.request_stop()
		coord.join(threads)
        
		if iteration % 20000 == 0:
			save_path = saver.save(sess, "/home/dl/Documents/Shamane Research - Repo/tensorflow-seq2seq-tutorials-master/check",global_step=iteration)
			print("Model saved in path: %s" % save_path)
            


In [ ]:
predict = tf.argmax(logits, axis=1)
predict
batchLoss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=batch_label)
loss = tf.reduce_mean(batchLoss)
t_vars = tf.trainable_variables()



#tf.initialize_all_variables()
checkpoint_dir='/home/dl/Documents/Shamane Research - Repo/tensorflow-seq2seq-tutorials-master/check'
saver = tf.train.Saver()
tf.initialize_all_variables()
with tf.Session() as sess:
	tf.train.start_queue_runners()
#	sess.run(tf.local_variables_initializer())
#	tf.global_variables_initializer().run()

	coord = tf.train.Coordinator()
	threads = tf.train.start_queue_runners(coord=coord)    
	ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    
	if ckpt and ckpt.model_checkpoint_path:
		print(ckpt.model_checkpoint_path)
		if os.path.isabs(ckpt.model_checkpoint_path):
			saver.restore(sess, ckpt.model_checkpoint_path)
		else:
			print("zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzgdgdgd")
			saver.restore(sess, os.path.join(checkpoint_dir,ckpt.model_checkpoint_path))
		global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
		print('Successfully loaded model from %s at step=%s.' %(ckpt.model_checkpoint_path, global_step))



	for iteration in xrange(100000):
		print("Iteration NUmber :", iteration)        
		_trainY, _predict = sess.run([batch_label, predict])
        
		        

		_label = np.argmax(_trainY, axis=1)
		_accuracy = np.mean(_label == _predict)        
		print(_accuracy)
		coord.request_stop()
		coord.join(threads)
        

            

Instructions for updating:
Use `tf.global_variables_initializer` instead.
/home/dl/Documents/Shamane Research - Repo/tensorflow-seq2seq-tutorials-master/check-20000
INFO:tensorflow:Restoring parameters from /home/dl/Documents/Shamane Research - Repo/tensorflow-seq2seq-tutorials-master/check-20000
Successfully loaded model from /home/dl/Documents/Shamane Research - Repo/tensorflow-seq2seq-tutorials-master/check-20000 at step=20000.
('Iteration NUmber :', 0)
0.125
('Iteration NUmber :', 1)
0.0625
('Iteration NUmber :', 2)
0.125
('Iteration NUmber :', 3)
0.0
('Iteration NUmber :', 4)
0.0625
('Iteration NUmber :', 5)
0.0625
('Iteration NUmber :', 6)
0.0625
('Iteration NUmber :', 7)
0.0625
('Iteration NUmber :', 8)
0.0
('Iteration NUmber :', 9)
0.0625
('Iteration NUmber :', 10)
0.0625
('Iteration NUmber :', 11)
0.0625
('Iteration NUmber :', 12)
0.0
('Iteration NUmber :', 13)
0.0625
('Iteration NUmber :', 14)
0.125
('Iteration NUmber :', 15)
0.0625
('Iteration NUmber :', 16)
0.0625
('Iterati